<a href="https://colab.research.google.com/github/NeutrinoExplorer/iQuHACK2025/blob/main/ClassiqChallenge/Classiq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
!pip install classiq

# Import necessary libraries
from classiq import qfunc, QNum, hadamard_transform, control, RY, QBit
from classiq.execution import execute
from classiq.modeling import create_model, ExecutionPreferences, Constraints, Preferences
from classiq.backends import ClassiqBackendPreferences, ClassiqSimulatorBackendNames
from classiq.visualization import show

# Function to create the Gaussian state preparation
def create_solution(resolution: int):
    fraction_digits = resolution - 2
    EXP_RATE = 1

    @qfunc
    def prepare_gaussian(x: QNum):
        # Step 1: Apply Hadamard transform to create superposition
        hadamard_transform(x)

        # Step 2: Apply controlled rotations to create the Gaussian distribution
        for i in range(x.size):
            control(x[i], lambda: RY(2 * EXP_RATE * (2 ** (-i - 1)), x[i]))

    return prepare_gaussian

# Function to create the quantum program
def create_qprog(prepare_gaussian_function, resolution: int, num_shots: int=1, optimization_parameter: str="CX", stage=int):
    fraction_digits = resolution - 2

    @qfunc
    def main(x: QNum[resolution]):
        # Allocate the quantum variable x with the specified resolution
        allocate(x.size, x)

        # Prepare Gaussian state
        prepare_gaussian_function(x)

    # Backend preferences for the state vector simulator
    backend_preferences = ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    )

    # Generate the quantum model based on the specified stage
    if stage == 1:
        qmod = create_model(
            main,
            execution_preferences=ExecutionPreferences(
                num_shots=num_shots, backend_preferences=backend_preferences
            ),
            constraints=Constraints(
                max_width=18, optimization_parameter=optimization_parameter
            ),
        )
    elif stage == 2:
        qmod = create_model(
            main,
            preferences=Preferences(timeout_seconds=1000),
            execution_preferences=ExecutionPreferences(
                num_shots=num_shots, backend_preferences=backend_preferences
            ),
            constraints=Constraints(
                max_width=127, optimization_parameter=optimization_parameter
            ),
        )
    else:
        return "Error: The `stage` variable should be set as an integer (1 or 2)."

    # Synthesize the quantum program
    qprog = synthesize(qmod)

    return qprog

# Creating the quantum program for stage 1
resolution_stage_1 = 8  # Fixed resolution
qprog_stage_1 = create_qprog(create_solution(resolution_stage_1), resolution_stage_1, stage=1)

# Display the quantum circuit
show(qprog_stage_1)

# Execute the quantum program
result_stage_1 = execute(qprog_stage_1).get_sample_result()

# Output the parsed state vector
print(result_stage_1.parsed_state_vector)
